# Summary
This document corresponds to Exercise 4 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Exercises/Course.pdf).

The first step consists in installing [PEPit](https://pypi.org/project/PEPit/) and its dependencies.

In [ ]:
!pip install pepit

In [ ]:
from PEPit import PEP
from PEPit.functions import ConvexFunction
from PEPit.primitive_steps import proximal_step

def wc_proximal_point(gamma, n, verbose=1):
    
    # Instantiate PEP
    problem = PEP()

    # Declare a convex function
    func = problem.declare_function(ConvexFunction)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and x^*
    problem.set_initial_condition((x0 - xs) ** 2 <= 1)

    # Run n steps of the proximal point method
    x = x0
    for _ in range(n):
        x, _, fx = proximal_step(x, func, gamma)

    # Set the performance metric to the final distance to optimum in function values
    problem.set_performance_metric(fx - fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose,dimension_reduction_heuristic="trace")

    # Return the worst-case guarantee of the evaluated method 
    return pepit_tau, (x-xs).eval()

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

n_max = 30
n_list = list(range(1,n_max))
gamma = 1
verbose = 0

pepits_taus_ppm = list()

for i in range(n_max-1):
    t0= time.process_time()
    pepit_tau,_ = wc_proximal_point(gamma, i+1, verbose)
    pepits_taus_ppm.append(pepit_tau)
    t1 = time.process_time() - t0
    print(i+1, '/', n_max-1,' done (elapsed time:',"%.2f" %t1,'[s])')

In [ ]:
plt.plot(n_list, pepits_taus_ppm, '-')

plt.xlabel('N')
plt.ylabel('f(x_i)-f_\star')
plt.loglog()
plt.show()